In [8]:
import tensorflow as tf
import os

In [9]:
gpus = tf.config.experimental.list_physical_devices("GPUS")
len(gpus)

0

In [11]:
import cv2
import imghdr

C:\Users\User\AppData\Local\Temp\ipykernel_23816\4232469594.py:2: DeprecationWarning: 'imghdr' is deprecated and slated for removal in Python 3.13
  import imghdr


In [16]:
data_dir = 'data'
# os.listdir(os.path.join(data_dir,'images_sad'))
os.listdir(data_dir)

['images_happy', 'images_sad']

In [13]:
image_exts = ['jpeg','jpg','bmp','png']
image_exts

['jpeg', 'jpg', 'bmp', 'png']

REMOVE DODGY IMAGES

In [18]:
for image_class in os.listdir(data_dir):
    for image in os.listdir(os.path.join(data_dir, image_class)):
        image_path = os.path.join(data_dir,image_class,image)
        try:
            img = cv2.imread(image_path)
            tip = imghdr.what(image_path)
            if tip not in image_exts:
                print("Image not in ext list {}".format(image_path))
                os.remove(image_path)
        except Exception as e:
            print('Issue with image {}'.format(image_path))

Image not in ext list data\images_happy\427394C200000578-0-image-a-24_1500384481554.jpg
Image not in ext list data\images_happy\depositphotos_391577690-stock-photo-cheerful-woman-outstretched-hands-sitting.jpg
Image not in ext list data\images_happy\README.md
Image not in ext list data\images_happy\traitshappypeople.jpg
Image not in ext list data\images_sad\depositphotos_217564870-stock-photo-young-attractive-sad-black-african.jpg
Image not in ext list data\images_sad\depositphotos_4584891-stock-photo-sad-man-in-a-empty.jpg
Image not in ext list data\images_sad\sad-young-woman-feeling-stressed-footage-107160392_iconl.jpeg
